# Web Scraping

### Importar librerias

In [25]:
import requests
import random
import pandas as pd
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

### Recupera todos los enlaces de artículos de una página específica (por mes)

In [26]:
def get_links(url):
    
    try:
        ua = UserAgent()
        headers = {"User-Agent" : ua.random}
        data = requests.get(url, headers=headers)
        soup = BeautifulSoup(data.text, "html5lib")
        
        all_links = []
        
        contents = soup.find_all("div", class_="single-blog-content")
        
        for content in contents:
            links = content.find_all("a", target = "_blank")
        
            for link in links:
                all_links.append(link.get("href"))
        
        return all_links
    
    except requests.exceptions.RequestException as e:
        print(f"Error en la URL {url}: {e}")
        return []

### Construye la URL de la página correspondiente al mes y llama a get_links() para obtener los enlaces de artículos publicados en ese mes.

In [27]:
def find_all_links_in_month(month):
    url = "https://www.lajornadamaya.mx/k'iintsil/{}-2021".format(month)
    return get_links(url)

###  Extrae la información de un artículo a partir de su enlace, extrayendo Fecha - Titulo - Contenido

In [28]:
def get_info(link):
    
    try:
        ua = UserAgent()
        headers = {"User-Agent" : ua.random}
        data = requests.get(link, headers=headers)
        soup = BeautifulSoup(data.text, "html5lib")
        
        # Busqueda del titulo
        try:
            title = soup.find("h1").get_text(strip=True) 
        except AttributeError:
            print(f"Título no encontrado: {link}")
            title = "Sin Titulo"
        # Opcional: Busqueda de la ubicacion
        # Busqueda de la fecha
        try:
            info = soup.find("div", class_="post-meta")
            if not info:
                raise ValueError("Informacion no encontrada")
            
            date_pattern = r"\d{2}/\d{2}/\d{4}"
            date_match = re.search(date_pattern, info.get_text())
            if not date_match:
                raise ValueError("Fecha no encontrada")
             
            date = date_match.group().strip() 
            
        except (AttributeError, ValueError) as e:
            print(f"Error al extraer la fecha: {link}")
            date = "Fecha desconocida"
        
        # Busqueda del contenido
        try:
            article = info.find_next("p").find_next("p")
            text = ""
            if not article:
                raise ValueError("Articulo no encontrado")
            
            paragraphs = article.find_all_next("p")
            for paragraph in paragraphs :
                if not paragraph.find("strong"):
                    text += paragraph.get_text() + "\n"
                
        except (AttributeError, ValueError) as e:
            print(f"Error al extraer el articulo: {link}")
            text = "Contenido no disponible"
            
        # Opcional: Busqueda del autor
        # Opcional: Busqueda del traductor
        
        return {
            "Fecha": date,
            #"Author": author,
            #"Traductor": traductor,
            #"Ubicacion": location,
            "Titulo": title,
            "Contenido": text.strip()
        }
         
    except requests.exceptions.RequestException as e:
        print(f"Error en la URL {link}: {e}")
        return None

### Procesa el scraping para cada mes del año
 - Recuperando los enlaces y los detalles de cada artículo
 - Guardándolos en un archivo .csv llamado datos.csv.

In [29]:
def scraping():
    months = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]
    df = pd.DataFrame()
    dfs = []
    for month in months:
        all_links = find_all_links_in_month(month)
        
        data_month = []
    
        print(f"Mes:{month}, Total: {len(all_links)} \n")
        
        for link in all_links:
            if link: 
                info = get_info(link)
                if info:
                    data_month.append(info)
            
        df_month = pd.DataFrame(data_month)
        dfs.append(df_month)
        
    df = pd.concat(dfs, ignore_index=True)
    df.to_csv("datos.csv", index=False, encoding='utf-8')
       


In [30]:
scraping()

Mes:enero, Total: 55 

Mes:febrero, Total: 51 

Mes:marzo, Total: 61 

Mes:abril, Total: 54 

Mes:mayo, Total: 59 

Mes:junio, Total: 64 

Mes:julio, Total: 67 

Mes:agosto, Total: 66 

Mes:septiembre, Total: 56 

Mes:octubre, Total: 68 

Mes:noviembre, Total: 66 

Mes:diciembre, Total: 52 



# Deteccion de maya usando enegramas 

In [ ]:
f

Idioma detectado: maya
Similitudes: {'maya': 0.7619047619047619, 'es': 0.7142857142857143}
Counter({'al ': 187, ' ca': 174, 'il ': 167, 'in ': 163, ' tu': 159, 'ch ': 157, 'an ': 118, ' ti': 109, 'ech': 107, 'el ': 106, 'ix ': 106, ' in': 103, ' la': 98, ' ci': 97, 'tu ': 92, ' ù ': 91, ' ya': 90, 'ob ': 87, ' te': 86, 'l t': 83, ' ma': 83, 'ic ': 82, 'ti ': 82, 'ich': 82, 'ca ': 81, 'en ': 78, ' ú ': 76, 'ah ': 73, 'l c': 72, 'ay ': 70, ' à ': 70, 'lic': 69, 'hun': 68, 'n t': 66, 'bal': 65, ' ta': 65, ' ka': 65, 'tec': 62, ' ua': 60, 'eba': 59, 'cil': 59, 'tic': 58, ' ba': 58, 'ili': 56, 'n c': 55, 'ma ': 55, 'h t': 55, 'xan': 54, 'lay': 52, 'tan': 52, ' cu': 51, 'ac ': 49, 'bil': 48, 'chi': 48, 'ul ': 47, 'zil': 47, 'a c': 47, 'hil': 46, 'aix': 46, 'kin': 45, ' yu': 44, 'l y': 43, ' pa': 43, ' ki': 43, ' na': 42, 'lal': 42, 'e c': 42, ' ix': 42, ' he': 41, 'tzi': 41, 'men': 41, 'ten': 41, 'a t': 40, ' ah': 40, ' hu': 40, 'ppe': 40, ' yo': 40, ' zi': 40, ' dz': 39, 'pel': 39, 'il\n': 